In [98]:
import pandas as pd
import numpy as np
import openpyxl
import re
from tqdm import tqdm
import os

In [88]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WhitespaceTokenizer
# from nltk.tokenize import WordPunctTokenizer

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.tag import pos_tag

from nltk.stem import WordNetLemmatizer

In [87]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/kimmonica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kimmonica/nltk_data...


True

### def

In [104]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [ ]:
def clean_words(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^0-9]+' 
        text = re.sub(pattern=pattern, repl='', string=text) 
        text = int(text)
    except:
        text = '숫자없음'
    return text 

#### dataset

In [101]:
df_path = '/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/dataset/'
file_list = os.listdir(df_path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')]

df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_excel(df_path + i)
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)

,Unnamed: 0,movie_cnt,movie_title,age,reviews
0,0,245번,Andi Mack,Child,-Why do most of the kids in my school like thi...
1,1,245번,Andi Mack,Child,-that show is rated TV-MA
2,2,245번,Andi Mack,Child,"-I haven't even watched the first episodes, bu..."
3,3,245번,Andi Mack,Child,"-Brah, this is possibly the worst TV show I ha..."
4,4,245번,Andi Mack,Child,-So everyone is saying that everything about t...
...,...,...,...,...,...
273128,4995,245번,Andi Mack,Child,-So everyone is saying that everything about t...
273129,4996,245번,Andi Mack,Child,-( Note: I’m sorry this review is so long. I j...
273130,4997,245번,Andi Mack,Child,-This show is amazing!! I watched this show wh...
273131,4998,245번,Andi Mack,Child,-its just poorly done


#### data loading

In [5]:
df = pd.read_csv('/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/moview_reviews.csv', index_col= 0)
df.head()

,movie_cnt,movie_title,age,reviews
0,0번,Alive,Child,Gore but not too crazy
1,0번,Alive,Child,If you dont get scared easily then you should ...
2,0번,Alive,Child,This movie is about an adult that plays videog...
3,0번,Alive,Child,its a really good movie keeps you interested ...
4,0번,Alive,Child,I loved it but some other teens might be sensi...


In [30]:
data = df.copy()

In [17]:
def clean_words(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^0-9]+' 
        text = re.sub(pattern=pattern, repl='', string=text) 
        text = int(text)
    except:
        text = '숫자없음'
    return text 

In [21]:
df['age'].value_counts()

Child                 89858
Teen 13 years old     19916
Teen 14 years old     10403
Kid 11 years old       9151
Kid 12 years old       7636
Kid 10 years old       6975
Teen 15 years old      5309
Kid 9 years old        4564
Kid 8 years old        2553
Teen 16 years old      2108
Teen 17 years old      1364
Kid 7 years old         954
Kid 6 years old         504
Adult 18 years old      201
Kid 0 years old         189
Kid 2 years old          74
Kid 5 years old          73
Kid 3 years old          50
Kid 4 years old          47
Kid 1 years old           4
Name: age, dtype: int64

In [24]:
data['age'].value_counts()

숫자없음    89858
13      19916
14      10403
11       9151
12       7636
10       6975
15       5309
9        4564
8        2553
16       2108
17       1364
7         954
6         504
18        201
0         189
2          74
5          73
3          50
4          47
1           4
Name: age, dtype: int64

In [31]:
data['age'] = data['age'].apply(clean_words)

In [34]:
data= data[data['age']!='숫자없음']
data['age'].value_counts()

13    19916
14    10403
11     9151
12     7636
10     6975
15     5309
9      4564
8      2553
16     2108
17     1364
7       954
6       504
18      201
0       189
2        74
5        73
3        50
4        47
1         4
Name: age, dtype: int64

In [35]:
data['age'] = data['age'].astype(int)

In [39]:
data = data[data['age']<13]
data = data[data['age']>5]
data.age.value_counts()

11    9151
12    7636
10    6975
9     4564
8     2553
7      954
6      504
Name: age, dtype: int64

In [40]:
data.to_csv('child_data.csv')

In [42]:
data.head()

,movie_cnt,movie_title,age,reviews
54,7번,1 Angry Black Man,7,This is not for 10 year olds
55,7번,1 Angry Black Man,7,The title makes me laughing XD This movie is t...
211,12번,12 Years a Slave,9,What a beautiful film 12 Years a Slave is Thi...
212,12번,12 Years a Slave,9,This movie shows how horrible slavery was and ...
213,12번,12 Years a Slave,9,Im 11 I watched this with my mother and I thou...


In [ ]:
def tokenize(df): 
    tokenizer = WhitespaceTokenizer()
    processed_data = [] 
    for sent in tqdm(data['reviews']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data

In [57]:
f = open('reviews_tokenize.txt','w')

In [86]:
k = pos_tag(word_tokenize("John's big idea isn't all that bad.")) 
print(k)
for i in k :
    if i =='POS'(i[1])

[('John', 'NNP'), ("'s", 'POS'), ('big', 'JJ'), ('idea', 'NN'), ('is', 'VBZ'), ("n't", 'RB'), ('all', 'PDT'), ('that', 'DT'), ('bad', 'JJ'), ('.', '.')]
NNP
POS
JJ
NN
VBZ
RB
PDT
DT
JJ
.


In [92]:
tokenizer = sent_tokenize()
tokenizer(data['reviews'].tolist()[2])

TypeError: sent_tokenize() missing 1 required positional argument: 'text'

In [96]:
test = data['reviews'].tolist()[2]
lensent_tokenize(test))

SyntaxError: unmatched ')' (79857623.py, line 2)

In [89]:
for i in data['reviews'].tolist()[:10]:
    tokenizer = WhitespaceTokenizer()
    
    print(pos_tag(tokenizer.tokenize(i)))

[('This', 'DT'), ('is', 'VBZ'), ('not', 'RB'), ('for', 'IN'), ('10', 'CD'), ('year', 'NN'), ('olds', 'NNS')]
[('The', 'DT'), ('title', 'NN'), ('makes', 'VBZ'), ('me', 'PRP'), ('laughing', 'VBG'), ('XD', 'RBR'), ('This', 'DT'), ('movie', 'NN'), ('is', 'VBZ'), ('trash', 'JJ'), ('and', 'CC'), ('racist', 'NN')]
[('What', 'WP'), ('a', 'DT'), ('beautiful', 'JJ'), ('film', 'NN'), ('12', 'CD'), ('Years', 'VBZ'), ('a', 'DT'), ('Slave', 'NNP'), ('is', 'VBZ'), ('This', 'DT'), ('is', 'VBZ'), ('just', 'RB'), ('a', 'DT'), ('phenomenal', 'JJ'), ('movie', 'NN'), ('12', 'CD'), ('Years', 'VBZ'), ('a', 'DT'), ('Slave', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('movie', 'NN'), ('thats', 'VBZ'), ('very', 'RB'), ('inspiring', 'JJ'), ('and', 'CC'), ('filled', 'VBN'), ('with', 'IN'), ('great', 'JJ'), ('performances', 'NNS'), ('Ill', 'NNP'), ('talk', 'NN'), ('about', 'IN'), ('the', 'DT'), ('performances', 'NNS'), ('first', 'JJ'), ('Chiwetel', 'NNP'), ('Ejiofor', 'NNP'), ('gives', 'VBZ'), ('the', 'DT'), ('best', 'JJ

In [76]:
tokenizer = WhitespaceTokenizer()
tokenizer.tokenize(test)

['This', 'is', 'not', 'for', '10', 'year', 'olds']